In [6]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# 加载数据
data = pd.read_csv('problem4.csv')

# 检查数据格式
print(data.head())

# 计算指数加权协方差矩阵
def exponential_covariance(data, lambda_ewma=0.94):
    """
    计算指数加权协方差矩阵
    参数:
        data (DataFrame): 每列为资产的收益数据
        lambda_ewma (float): 指数加权因子
    返回:
        cov_matrix (numpy array): 指数加权协方差矩阵
    """
    returns = data.values
    weights = (1 - lambda_ewma) * lambda_ewma ** np.arange(len(returns)-1, -1, -1)
    weights /= np.sum(weights)  # 标准化权重
    mean_returns = np.average(returns, axis=0, weights=weights)
    deviations = returns - mean_returns
    cov_matrix = np.dot(weights * deviations.T, deviations)
    return cov_matrix

# 计算协方差矩阵
lambda_ewma = 0.94
cov_matrix = exponential_covariance(data, lambda_ewma)
print("指数加权协方差矩阵:")
print(cov_matrix)


         x1        x2        x3        x4        x5
0  0.085766 -0.118260 -0.010997 -0.000582 -0.009273
1 -0.127694  0.287384  0.205569 -0.098169  0.115423
2  0.062381  0.074674  0.042317 -0.031115  0.060666
3  0.080215 -0.159533 -0.082036  0.071518 -0.073672
4 -0.093351  0.046987 -0.056058  0.003045 -0.014686
指数加权协方差矩阵:
[[ 0.00800778 -0.001406   -0.00417763  0.00181268  0.00055517]
 [-0.001406    0.00532315  0.00295607 -0.00223105  0.00270149]
 [-0.00417763  0.00295607  0.00583109 -0.00303737  0.00182102]
 [ 0.00181268 -0.00223105 -0.00303737  0.00285228 -0.00215035]
 [ 0.00055517  0.00270149  0.00182102 -0.00215035  0.00244845]]


# a. What are the risk parity portfolio weights using standard deviation as your risk measure? (

In [7]:
# 4a. 计算风险平价组合权重（基于标准差）
def risk_parity_weights_std(cov_matrix):
    """
    使用标准差作为风险测量计算风险平价组合权重
    参数:
        cov_matrix (numpy array): 协方差矩阵
    返回:
        weights (numpy array): 风险平价组合权重
    """
    std_devs = np.sqrt(np.diag(cov_matrix))  # 每个资产的标准差
    inverse_std = 1 / std_devs  # 风险的倒数
    weights = inverse_std / np.sum(inverse_std)  # 归一化权重
    return weights

weights_std = risk_parity_weights_std(cov_matrix)
print("基于标准差的风险平价组合权重:")
print(weights_std)


基于标准差的风险平价组合权重:
[0.1452979  0.17820953 0.17027101 0.24345536 0.2627662 ]


# b. What are the risk parity portfolio weights using expected shortfall as your risk measure?

In [8]:
# 4b. 计算风险平价组合权重（基于期望短缺）
def calculate_es_std(mean=0, std=1, alpha=0.05):
    """
    计算正态分布的期望短缺（ES）
    参数:
        mean (float): 均值
        std (float): 标准差
        alpha (float): 置信水平
    返回:
        es (float): 期望短缺值
    """
    var = norm.ppf(alpha, loc=mean, scale=std)
    es = mean - std * norm.pdf(norm.ppf(alpha)) / alpha
    return -es

def risk_parity_weights_es(cov_matrix, alpha=0.05):
    """
    使用期望短缺 (ES) 作为风险测量计算风险平价组合权重
    参数:
        cov_matrix (numpy array): 协方差矩阵
        alpha (float): 置信水平
    返回:
        weights (numpy array): 风险平价组合权重
    """
    std_devs = np.sqrt(np.diag(cov_matrix))  # 每个资产的标准差
    es_values = np.array([calculate_es_std(std=std, alpha=alpha) for std in std_devs])  # 每个资产的 ES
    inverse_es = 1 / es_values  # ES 的倒数
    weights = inverse_es / np.sum(inverse_es)  # 归一化权重
    return weights

weights_es = risk_parity_weights_es(cov_matrix, alpha=0.05)
print("基于期望短缺 (ES) 的风险平价组合权重:")
print(weights_es)


基于期望短缺 (ES) 的风险平价组合权重:
[0.1452979  0.17820953 0.17027101 0.24345536 0.2627662 ]
